# 토큰화

In [1]:
!pip install gluonnlp pandas tqdm
!pip install mxnet

!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2

#PyTorch불러오는거
!pip install torch
!pip install --upgrade torch torchvision

# KoBERT 토크나이저& 본체
!pip install git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hfpip
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 15.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595730 sha256=98a16e7ddbbb45ce781f83397fd01ba11e33ccbc81f4072facd7a9f0a6a03729
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 202 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 15.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.p

In [2]:
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np

from tqdm import tqdm, tqdm_notebook

from kobert import get_pytorch_kobert_model
from kobert import get_tokenizer

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

device = torch.device("cuda:0")
model, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [4]:
from google.colab import drive
drive.mount('/content/drive')

data=pd.read_excel('/content/drive/MyDrive/hackerthon/웰니스 대화 스크립트 데이터셋/02)웰니스_대화_스크립트_데이터셋.xlsx')

Mounted at /content/drive


In [5]:
from google.colab import drive
drive.mount('/content/drive')

data=pd.read_excel('/content/drive/MyDrive/hackerthon/웰니스 대화 스크립트 데이터셋/02)웰니스_대화_스크립트_데이터셋.xlsx')

posneg_data=[]
for i in data['utterance(긍정)'].unique():
  if(not pd.isnull(i)):
    posneg_data.append([i,'1'])
for i in data['utterance(부정)'].unique():
  if(not pd.isnull(i)):
    posneg_data.append([i,'0'])

print(posneg_data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[['응', '1'], ['어', '1'], ['엉', '1'], ['예스', '1'], ['네', '1'], ['그래', '1'], ['맞아', '1'], ['그런 거 같아', '1'], ['많아', '1'], ['있어', '1'], ['있었어요', '1'], ['있었던 것 같아', '1'], ['있었어', '1'], ['요새 그래', '1'], ['최근에 그래', '1'], ['요즘 그런 것 같아', '1'], ['많았어', '1'], ['기분이 안 좋아지는 것 같아', '1'], ['요즘에 그래', '1'], ['임신하고 나서 좀 그런 것 같아', '1'], ['맞는 거 같아', '1'], ['처져', '1'], ['처지는 것 같아', '1'], ['처져요', '1'], ['맞아 내가 딱 그래', '1'], ['요새 자꾸 처져', '1'], ['계속 그런 것 같아', '1'], ['어 엄청', '1'], ['임신하고 나선 계속 그러네', '1'], ['그렇다니까', '1'], ['엄청 처져', '1'], ['맨날 처지는 것 같아', '1'], ['좋아지지 않아', '1'], ['안 좋아', '1'], ['그런가 봐', '1'], ['안 좋아져', '1'], ['안 좋아요', '1'], ['맨날 그래', '1'], ['기분이 안 좋아져', '1'], ['응 항상 그래', '1'], ['요즘 기분이 안 좋아', '1'], ['임신하고 나선 계속 이래', '1'], ['기분이 안 좋아', '1'], ['늘 기분이 다운 돼', '1'], ['응 안 좋아', '1'], ['노력해도 기분이 영..', '1'], ['응 엄청', '1'], ['임신하고 나선 계속 그래', '1'], ['뿌예', '1'], ['뿌연 것 같아', '1'], ['

In [6]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        
        temp_list=[]
        for i in dataset:
          temp_list.append(transform([i[sent_idx]]))
        self.sentences = temp_list
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [7]:
#KoBERT 다중분류기 클래스
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [8]:
#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [9]:
#문장이 주어지면 예측하는 함수 정의
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 1:
                test_eval.append("긍정")
            elif np.argmax(logits) == 0:
                test_eval.append("부정")

        print(">> 현재 화자는 질문에 " + test_eval[0] + "합니다.")

In [10]:
# Setting parameters
max_len = 64 #  허용가능한 최대 문장길이 (토큰화 이후)
batch_size = 64 # 배치 크기 - 50개 데이터에서 배치가 10이면 5번의 반복으로 모든 데이터를 볼수있음
warmup_ratio = 0.1 # 전체학습중 warmup step의 비율 - warmup step은 수치적 안정성을 위해 학습초반 조금더 작은 learning_rate를 적용하는 것을의미함
num_epochs = 20 # 전체데이터 학습시키는 횟수
num_workers = 5 # 데이터를 로딩하는데 사용할 멀티프로세스 수
max_grad_norm = 1 
log_interval = 200
learning_rate =  5e-5

In [11]:
#train set, test set 나누기기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(posneg_data, test_size=0.2, random_state=0)

In [12]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [13]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [14]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=num_workers)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=num_workers)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [15]:
#BERT 모델 불러오기
model = BERTClassifier(model,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다항항분류를 위한 대표적인 loss func
#loss_fn = nn.BCELoss() # 이항분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [16]:
# 학습시키기

train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    #학습모드
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        #print(label.shape,out.shape)
        #m=nn.Softmax(dim=1)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    #평가모드
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

    
    torch.save(model,f'/content/drive/MyDrive/hackerthon/긍정부정모델-우빈/model{e}.pt')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/26 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.6590288877487183 train acc 0.5625
epoch 1 train acc 0.5462740384615384


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/7 [00:00<?, ?it/s]

...

epoch 19 test acc 0.9642857142857143


  0%|          | 0/26 [00:00<?, ?it/s]

epoch 20 batch id 1 loss 0.013676022179424763 train acc 0.984375
epoch 20 train acc 0.9975961538461539


  0%|          | 0/7 [00:00<?, ?it/s]

epoch 20 test acc 0.9642857142857143


In [17]:
import plotly.express as px
import plotly.graph_objects as go
 
x=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
fig = go.Figure() # Figure 추가
fig.add_traces(go.Scatter(x=x, y=train_history, name='Train Accuracy', line = dict(color='royalblue', width=4))) # 선 그래프 추가.
fig.add_traces(go.Scatter(x=x, y=test_history, name='Test Accuracy', line = dict(color='firebrick', width=4))) # 선 그래프 추가.
fig.show() # 그래프 출력 이 부분은 생략해도 그래프는 출력됨.

In [20]:
while True :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break    
    model=torch.load(f'/content/drive/MyDrive/hackerthon/긍정부정모델-우빈/model16.pt',map_location=device)
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 요즘은 그래
>> 현재 화자는 질문에 부정합니다.


하고싶은 말을 입력해주세요 : 요즘 좀 그래
>> 현재 화자는 질문에 긍정합니다.


하고싶은 말을 입력해주세요 : 예전엔 그랬어
>> 현재 화자는 질문에 긍정합니다.


하고싶은 말을 입력해주세요 : 앞으로도 그럴것같아
>> 현재 화자는 질문에 긍정합니다.


하고싶은 말을 입력해주세요 : 0
